#### Arcpy + Geopandas combination

In [1]:
#Must be opened from ArcGIS Python Command Prompt (juptyter lab)
# Must have signed into ArcGIS online? (run ArcGIS Pro)

import arcpy
from arcpy import env
from arcpy.sa import *

import pandas as pd
import geopandas as gpd

from pathlib import Path
from datetime import datetime

In [2]:
def renameShapeField(inshp,infieldname,outfieldname):
    '''renames a field in a shapefile using geopandas because arcpy doesn't support this.'''
    ingdf = gpd.read_file(inshp)
    ingdf.rename(columns={infieldname: outfieldname}, inplace=True)
    ingdf.to_file(inshp)
    print(f'Fieldname {infieldname} changed to {outfieldname} in shapefile.')

In [52]:
#inputs
#DEMS
# dem1 = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\dems\orig\brandy_dem2019.tif")
# dem2 = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\dems\orig\brandy_dem2020.tif")
dem1 = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\dems\orig\brandy_dem2018.tif")
dem2 = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\dems\orig\brandy_dem2019.tif")

#Stable polygons
# stablepolyshp = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\shp\Stable_poly_19-20_expanded.shp")
stablepolyshp = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\shp\Stable_poly_18-19_expanded.shp")

#Output dod name stem
dodnamestem = r"DoD_19-18"
detrendnamestem = r"Detrend_19-18_polyn"

#output dir
demdiff_dir = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\demdiff19-18")
outdod_dir = Path(demdiff_dir, r"dod")
outdod_unadj_dir  = Path(demdiff_dir, r"dod\unadj")
outdod_adj_dir  = Path(demdiff_dir, r"dod\adj")
outdod_pt_dir  = Path(demdiff_dir, r"dod\shp")
outtrendraster_dir = Path(demdiff_dir, r"detrend")
outtrend_pt_dir = Path(demdiff_dir, r"detrend\shp")
scratch_dir = Path(demdiff_dir, r"arcpyscratch")

#create parent if doesn't exist
demdiff_dir.mkdir(parents=True, exist_ok=True)
#create subdir if don't exist
for direc in [outdod_dir, outdod_unadj_dir, outdod_adj_dir, outdod_pt_dir, outtrendraster_dir, outtrend_pt_dir, scratch_dir]:
    direc.mkdir(parents=True, exist_ok=True)

# # Set local variables
# instablepointfeatures = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\dod\shp\DoD_20-19_unadj_pts_stable.shp")

In [50]:
# Set environment settings
arcpy.env.workspace = str(scratch_dir)
arcpy.env.overwriteOutput = True
arcpy.env.compression = "LZW"

env.extent = str(dem1)
env.snapRaster = str(dem1)

# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial");

In [51]:
#Make unadjusted DoD
outunadjdod = Path(outdod_unadj_dir,dodnamestem + r'_unadj.tif')

dod = RasterCalculator([str(dem1), str(dem2)], ["dem1", "dem2"],
                                       "dem2-dem1", "FirstOf", "FirstOf")

dod.save(str(outunadjdod))


In [ ]:
#Convert DoD to point shapefile, then clip with stable polygons and rename field to 'dod_unadj'
outstablept_shp = str(Path(outdod_pt_dir, outunadjdod.stem + r'_pts.shp')) #same as DoD, but '_pts.shp')

arcpy.RasterToPoint_conversion(dod, r"memory\tempRasPt", "VALUE");
arcpy.Clip_analysis(r"memory\tempRasPt", str(stablepolyshp), outstablept_shp, "");
#rename field
renameShapeField(outpointshape,'grid_code','dod_unadj')


In [75]:
#Calculate trend surface using all stable points (polynomial 0, 1 and 2)

#Inputs/outputs
stablept_shp = outstablept_shp #(from cell above)
outtrendrasterstem = str(Path(outtrendraster_dir, detrendnamestem))

zField = "dod_unadj"
cellSize = 0.25
# PolynomialOrder = 2 (set in loop)
regressionType = "LINEAR"

for polyn_order in [0, 1, 2]:
    # Execute Trend
    print(f'Creating trend surface polynomial order {polyn_order}...')
    outTrend = Trend(stablept_shp, zField, cellSize, 
                     polyn_order, regressionType)
    outTrend.save(outtrendrasterstem + str(polyn_order) + ".tif")


Creating trend surface polynomial order 0...
Creating trend surface polynomial order 1...
Creating trend surface polynomial order 2...


In [97]:
# '''
# Apply to DoD to evaluate visually
# Adjustment order is:
#     DoD - TrendRaster = AdjustedDoD
# This is equivalent to adjusting DEM1:
#      DEM2 - DEM1 - TrendRaster = AdjustedDoD
# Since TrendRaster = DoD residual which should be zero.
# For example:
# DEM2 = 3, DEM1 = 5, DoD = -2, TrendRaster = -2
# AdjustedDoD =  DEM2 - DEM1 - TrendRaster = 0
#             =  3 - 5 -(-2) = 0
#             =  DoD - TrendRaster = 0
#             =  -2 -(-2) = 0
# '''

#inputs/outputs
inunadjdod = str(outunadjdod) #from above cell
intrendrasterstem = outtrendrasterstem #from cell above
outadjdodstem = str(Path(outdod_adj_dir, dodnamestem + r'_adj_polyn_'))

#loop through detrend surfaces and apply to DoD
for polyn_order in [0,1,2]:
    #apply adjustment
    trendraster = intrendrasterstem + str(polyn_order) + '.tif'
    adj = RasterCalculator([inunadjdod, trendraster], ["dod", "trendraster"],
                                       "dod-trendraster", "FirstOf", "FirstOf")
    adj.save(outadjdodstem + str(polyn_order) + '.tif')
    

In [78]:
#Extract residual values from each adjusted DoD trend surface and evaluate bulk points

stablept_shp = str(Path(outdod_pt_dir, outunadjdod.stem + r'_pts.shp')) #same as DoD, but '_pts.shp')
inadjdodstem = str(Path(outdod_adj_dir, dodnamestem + r'_adj_polyn_'))
outdetrendevalptshp = str(Path(outtrend_pt_dir, Path(outdod_pt_dir, outunadjdod.stem + r'_pts.shp') .stem + r'_detrend_eval.shp'))

#copy stable points to eval file
arcpy.Copy_management(stablept_shp, outdetrendevalptshp);

#derive list of adjusted dods and output fieldnames for multi-raster extract
inras_outfield_list = [[inadjdodstem + str(n) + '.tif', 'poly' + str(n) + 'resid'] for n in [0,1,2]]

#extract adjusted dod residual values to points
ExtractMultiValuesToPoints(outdetrendevalptshp, inras_outfield_list, "NONE")

# Evaluate residual values
#report    
for col in ['dod_unadj','poly0resid', 'poly1resid', 'poly2resid']:
    print(f'{col} mean = {str(round(tempgdf[col].mean(),7))} \n'
          f'{col} std = {str(round(tempgdf[col].std(),7))} \n'
          f'{col} max = {str(round(tempgdf[col].max(),7))} \n'
          f'{col} min = {str(round(tempgdf[col].min(),7))} \n'
          '\n'
         )

#clean up 
for i in [0,1,2]:
    arcpy.management.Delete(str(Path(outdetrendpath + '\\temp_detrendeval_' + str(i) + '.shp')));

Sampling raster D:\Whiskeytown\dem_diff\brandy_creek\dod\DOD_brandy_20-19_poly0adjust.tif ...
Renaming field.
Fieldname RASTERVALU changed to poly0resid in shapefile.
Sampling raster D:\Whiskeytown\dem_diff\brandy_creek\dod\DOD_brandy_20-19_poly1adjust.tif ...
Renaming field.
Fieldname RASTERVALU changed to poly1resid in shapefile.
Sampling raster D:\Whiskeytown\dem_diff\brandy_creek\dod\DOD_brandy_20-19_poly2adjust.tif ...
Renaming field.
Fieldname RASTERVALU changed to poly2resid in shapefile.
dod_unadj mean = 0.0415556 
dod_unadj std = 0.0587836 
dod_unadj max = 1.36026 
dod_unadj min = -0.3206482 


poly0resid mean = -0.0 
poly0resid std = 0.0587836 
poly0resid max = 1.3187044 
poly0resid min = -0.3622038 


poly1resid mean = -0.0 
poly1resid std = 0.051012 
poly1resid max = 1.3005401 
poly1resid min = -0.3364101 


poly2resid mean = 7.08e-05 
poly2resid std = 0.0492809 
poly2resid max = 1.2729789 
poly2resid min = -0.3268719 




### Decide which trend surface to use

In [45]:
# Evaluate Residual Error
# Subdivide stable polygons to use as test/train sets for residual error evaluation, then get poly and subdiv poly id onto points via intersection

#inputs
# target_subpoly_area = 9 # (9 = 144 points)size of subpolygons in m^2 (too small may result in overfitting of trend surface to validation points?)
target_subpoly_area = 100 # (100 = 1600 pts) size of subpolygons in m^2(too small may result in overfitting of trend surface to validation points?)
#points
inPointFeatures = r"D:\Whiskeytown\dem_diff\brandy_creek\dod\shp\DoD_19-18_unadj_pts_stable.shp"
outpointswithsubpolyid = r"D:\Whiskeytown\dem_diff\brandy_creek\dod\shp\DoD_19-18_unadj_pts_stable_100m_subpolyid.shp"
#polygons
stablepolyshp = r"D:\Whiskeytown\dem_diff\brandy_creek\shp\Stable_poly_18-19_expanded.shp"
outstablepolysubdivided = r"D:\Whiskeytown\dem_diff\brandy_creek\shp\Stable_poly_18-19_expanded_100m_subdivided.shp"

#subdivide polygon
arcpy.SubdividePolygon_management(
    stablepolyshp, outstablepolysubdivided, "EQUAL_AREAS","", target_subpoly_area, "", "", 
    "STACKED_BLOCKS")

#add subdiv poly id to poly attr table
polygdf = gpd.read_file(outstablepolysubdivided)
polygdf['subpolyid'] = polygdf.index
polygdf.to_file(outstablepolysubdivided)

#run spatial join to get 'subpolyid'
ptgdf = gpd.read_file(inPointFeatures)

#spatial join points with subdiv polygons to get polyid and subdividedpolyid on points for later test/train filter.
ptwithpolygdf = gpd.sjoin(ptgdf, polygdf, how="left", op='intersects')
#write output shp
ptwithpolygdf.to_file(outpointswithsubpolyid)

In [28]:
#iterate and do test/train splits to evaluate by subpolygon
print(datetime.now().strftime("%Y/%d/%m %H:%M:%S"))    
#number of iterations (~2-3 minutes each?)
num_loops = 100

#inputs
pointswithsubpolyid = r"D:\Whiskeytown\dem_diff\brandy_creek\dod\shp\DoD_19-18_unadj_pts_stable_100m_subpolyid.shp"
stablepolysubdivided = r"D:\Whiskeytown\dem_diff\brandy_creek\shp\Stable_poly_18-19_expanded_100m_subdivided.shp"

#load points with subpoly id into gdf to be filtered
ptwithpolygdf = gpd.read_file(pointswithsubpolyid)

#trend surface details
zField = "dod_unadj"
cellSize = 0.25
PolynomialOrder = 2
regressionType = "LINEAR"

#result df
resultsdf = pd.DataFrame(columns = ['bulk_pt_mean','bulk_pt_std','all_poly_mean','all_poly_std','paved_poly_mean','paved_poly_std','unpaved_poly_mean','unpaved_poly_std'])
#percent of poly to use for train 
trainfrac = 0.6

for i in range(num_loops):
    #clean up scratch dir first (leave results of final iteration in scratch to let user see them)
    for fn in [r"\temp_trainpoly.shp", r"\temp_valpoly.shp", r"\temp_trainpoint.shp", r"\temp_valpoint.shp", r"\temp_valpoint_withdetrend.shp",r"\temp_trendraster.tif"]:
        arcpy.management.Delete(arcpy.env.workspace + fn)
    print(f'Beginning iteration {i} ...')
    #subset poly to test/train
    arcpy.ga.SubsetFeatures(stablepolysubdivided, 
                            arcpy.env.workspace + r"\temp_trainpoly.shp", 
                            arcpy.env.workspace + r"\temp_valpoly.shp", 60, "PERCENTAGE_OF_INPUT")
    #load each to get poly id
    trainpolygdf = gpd.read_file(arcpy.env.workspace + r"\temp_trainpoly.shp")
    valpolygdf = gpd.read_file(arcpy.env.workspace + r"\temp_valpoly.shp")
    
    #use subpolyid to select train/val points
    trainptgdf = ptwithpolygdf[ptwithpolygdf['subpolyid'].isin(trainpolygdf['subpolyid'].tolist())]
    valptgdf = ptwithpolygdf[ptwithpolygdf['subpolyid'].isin(valpolygdf['subpolyid'].tolist())]
    
    #write to temp
    trainptgdf.to_file(arcpy.env.workspace + r"\temp_trainpoint.shp")
    valptgdf.to_file(arcpy.env.workspace + r"\temp_valpoint.shp")
    
    #create trend surface
    # Execute Trend
    print('Creating trend surface...')
    outTrend = Trend(arcpy.env.workspace + r"\temp_trainpoint.shp", zField, cellSize, 
                     PolynomialOrder, regressionType)
    outTrend.save(arcpy.env.workspace + r"\temp_trendraster.tif")
    
    #Sample raster on validate points
    print('Sampling trend surface...')
    ExtractValuesToPoints(arcpy.env.workspace + r"\temp_valpoint.shp", 
                          arcpy.env.workspace + r"\temp_trendraster.tif", 
                          arcpy.env.workspace + r"\temp_valpoint_withdetrend.shp",
                          "NONE", "VALUE_ONLY")
    
    #read into gdf to eval
    print('Evaluating trend surface...')
    evalgdf = gpd.read_file(arcpy.env.workspace + r"\temp_valpoint_withdetrend.shp")
    #apply to DoD value (dod_unadj - trendRasterValue)
    evalgdf['resid2'] = evalgdf['dod_unadj'] - evalgdf['RASTERVALU']
    
    #gather residuals grouped by polygons
    data = {'bulk_pt_mean': [evalgdf['resid2'].mean()],
            'bulk_pt_std': [evalgdf['resid2'].std()],
            'all_poly_mean': [evalgdf.groupby('subpolyid')['resid2'].mean().mean()],
            'all_poly_std': [evalgdf.groupby('subpolyid')['resid2'].mean().std()],
            'paved_poly_mean': [evalgdf[evalgdf['type'] == 'paved'].groupby('subpolyid')['resid2'].mean().mean()],
            'paved_poly_std': [evalgdf[evalgdf['type'] == 'paved'].groupby('subpolyid')['resid2'].mean().std()],
            'unpaved_poly_mean': [evalgdf[evalgdf['type'] == 'unpaved'].groupby('subpolyid')['resid2'].mean().mean()],
            'unpaved_poly_std': [evalgdf[evalgdf['type'] == 'unpaved'].groupby('subpolyid')['resid2'].mean().std()]
           }
    resultsdf = resultsdf.append(pd.DataFrame(data, 
                                              columns = ['bulk_pt_mean','bulk_pt_std','all_poly_mean','all_poly_std','paved_poly_mean','paved_poly_std','unpaved_poly_mean','unpaved_poly_std']), 
                                 ignore_index=True)
    
    
    print(datetime.now().strftime("%Y/%d/%m %H:%M:%S"))   
        
    #write intermediate output, overwrite to save progress
    resultsdf.to_csv(r"D:\Whiskeytown\dem_diff\brandy_creek\detrend_surface\ResidualSystematicErrorBy_100m_SubPolygon_19-18.csv")
    resultsdf.describe().to_csv(r"D:\Whiskeytown\dem_diff\brandy_creek\detrend_surface\ResidualSystematicErrorBy_100m_SubPolygon_SummaryStats_19-18.csv")



2021/28/01 17:09:32
Beginning iteration 0 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 17:12:06
Beginning iteration 1 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 17:14:26
Beginning iteration 2 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 17:16:45
Beginning iteration 3 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 17:19:06
Beginning iteration 4 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 17:21:27
Beginning iteration 5 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 17:23:47
Beginning iteration 6 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 17:26:07
Beginning iteration 7 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend sur

### Same as above, but using entire polygons instead of subidived polygons

In [36]:
#iterate and do test/train splits to evaluate by whole polygon
print(datetime.now().strftime("%Y/%d/%m %H:%M:%S"))    
#number of iterations (~2-3 minutes each?)
num_loops = 10
#inputs
pointswithsubpolyid = r"D:\Whiskeytown\dem_diff\brandy_creek\dod\shp\DoD_20-19_unadj_pts_stable_subpolyid.shp"
# stablepolysubdivided = r"D:\Whiskeytown\dem_diff\brandy_creek\shp\Stable_poly_19-20_expanded_subdivided.shp"
stablepolysubdivided = r"D:\Whiskeytown\dem_diff\brandy_creek\shp\Stable_poly_19-20_expanded.shp"

#load points with subpoly id into gdf to be filtered
ptwithpolygdf = gpd.read_file(pointswithsubpolyid)

#trend surface details
zField = "dod_unadj"
cellSize = 0.25
PolynomialOrder = 2
regressionType = "LINEAR"

#result df
resultsdf = pd.DataFrame(columns = ['bulk_pt_mean','bulk_pt_std','all_poly_mean','all_poly_std','paved_poly_mean','paved_poly_std','unpaved_poly_mean','unpaved_poly_std'])
#percent of poly to use for train 
trainfrac = 0.6

for i in range(num_loops):
    #clean up scratch dir first (leave results of final iteration in scratch to let user see them)
    for fn in [r"\temp_trainpoly.shp", r"\temp_valpoly.shp", r"\temp_trainpoint.shp", r"\temp_valpoint.shp", r"\temp_valpoint_withdetrend.shp",r"\temp_trendraster.tif"]:
        arcpy.management.Delete(arcpy.env.workspace + fn)
    print(f'Beginning iteration {i} ...')
    #subset poly to test/train
    arcpy.ga.SubsetFeatures(stablepolysubdivided, 
                            arcpy.env.workspace + r"\temp_trainpoly.shp", 
                            arcpy.env.workspace + r"\temp_valpoly.shp", 60, "PERCENTAGE_OF_INPUT")
    #load each to get poly id
    trainpolygdf = gpd.read_file(arcpy.env.workspace + r"\temp_trainpoly.shp")
    valpolygdf = gpd.read_file(arcpy.env.workspace + r"\temp_valpoly.shp")
    
    #use subpolyid to select train/val points
    trainptgdf = ptwithpolygdf[ptwithpolygdf['id'].isin(trainpolygdf['id'].tolist())]
    valptgdf = ptwithpolygdf[ptwithpolygdf['id'].isin(valpolygdf['id'].tolist())]
    
    #write to temp
    trainptgdf.to_file(arcpy.env.workspace + r"\temp_trainpoint.shp")
    valptgdf.to_file(arcpy.env.workspace + r"\temp_valpoint.shp")
    
    #create trend surface
    # Execute Trend
    print('Creating trend surface...')
    outTrend = Trend(arcpy.env.workspace + r"\temp_trainpoint.shp", zField, cellSize, 
                     PolynomialOrder, regressionType)
    outTrend.save(arcpy.env.workspace + r"\temp_trendraster.tif")
    
    #Sample raster on validate points
    print('Sampling trend surface...')
    ExtractValuesToPoints(arcpy.env.workspace + r"\temp_valpoint.shp", 
                          arcpy.env.workspace + r"\temp_trendraster.tif", 
                          arcpy.env.workspace + r"\temp_valpoint_withdetrend.shp",
                          "NONE", "VALUE_ONLY")
    
    #read into gdf to eval
    print('Evaluating trend surface...')
    evalgdf = gpd.read_file(arcpy.env.workspace + r"\temp_valpoint_withdetrend.shp")
    #apply to DoD value (dod_unadj - trendRasterValue)
    evalgdf['resid2'] = evalgdf['dod_unadj'] - evalgdf['RASTERVALU']
    
    #gather residuals grouped by polygons
    data = {'bulk_pt_mean': [evalgdf['resid2'].mean()],
            'bulk_pt_std': [evalgdf['resid2'].std()],
            'all_poly_mean': [evalgdf.groupby('id')['resid2'].mean().mean()],
            'all_poly_std': [evalgdf.groupby('id')['resid2'].mean().std()],
            'paved_poly_mean': [evalgdf[evalgdf['type'] == 'paved'].groupby('id')['resid2'].mean().mean()],
            'paved_poly_std': [evalgdf[evalgdf['type'] == 'paved'].groupby('id')['resid2'].mean().std()],
            'unpaved_poly_mean': [evalgdf[evalgdf['type'] == 'unpaved'].groupby('id')['resid2'].mean().mean()],
            'unpaved_poly_std': [evalgdf[evalgdf['type'] == 'unpaved'].groupby('id')['resid2'].mean().std()]
           }
    resultsdf = resultsdf.append(pd.DataFrame(data, 
                                              columns = ['bulk_pt_mean','bulk_pt_std','all_poly_mean','all_poly_std','paved_poly_mean','paved_poly_std','unpaved_poly_mean','unpaved_poly_std']), 
                                 ignore_index=True)
    
    
    print(datetime.now().strftime("%Y/%d/%m %H:%M:%S"))  
    
    #write intermediate output, overwrite to save progress
    resultsdf.to_csv(r"D:\Whiskeytown\dem_diff\brandy_creek\detrend_surface\ResidualSystematicErrorByWholePolygon_20-19.csv")
    resultsdf.describe().to_csv(r"D:\Whiskeytown\dem_diff\brandy_creek\detrend_surface\ResidualSystematicErrorByWholePolygon_SummaryStats_20-19.csv")



2021/28/01 13:29:39
Beginning iteration 0 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:31:19
Beginning iteration 1 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:34:18
Beginning iteration 2 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:35:43
Beginning iteration 3 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:37:11
Beginning iteration 4 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:39:40
Beginning iteration 5 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:42:17
Beginning iteration 6 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:45:16
Beginning iteration 7 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend sur